## Car 실습 - YOLO + Cascade

In [8]:
import numpy as np
import cv2

file_names = 'russia-cars.jpg'
img = cv2.imread(file_names)

car_num_name = './haar_files/haarcascade_russian_plate_number.xml'
cascade = cv2.CascadeClassifier()
cascade.load(cv2.samples.findFile(car_num_name))


min_confidence = 0.5
yolo_weight_file = './darknet/weights/yolov3.weights'
yolo_cfg_file = './darknet/cfg/yolov3.cfg'
yolo_name_file = './darknet/data/coco.names'

net = cv2.dnn.readNet(yolo_weight_file, yolo_cfg_file)

classes = []
with open(yolo_name_file, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_names = net.getUnconnectedOutLayersNames()

colors = np.random.uniform(0, 255, size=(len(classes) * 3, 3))

In [9]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.equalizeHist(gray)

In [10]:
height, width, channels = img.shape
blob = cv2.dnn.blobFromImage(img,
                            scalefactor=1/255.,
                            size=(416, 416),
                            mean=(0,0,0),
                            swapRB=True,
                            crop=False)

net.setInput(blob)
outs = net.forward(output_names)

classes_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]

        if confidence > min_confidence:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w//2) # start x
            y = int(center_y - h//2) # start y
            
            if class_id == 2:
                boxes.append([x, y, w, h])
                confidences.append(confidence.astype('float'))
                classes_ids.append(class_id)

# 감지된 모든 바운딩 박스
indexes = cv2.dnn.NMSBoxes(boxes, 
                          confidences,
                          min_confidence,
                          0.4) # IoU

for i in indexes:
    idx = i[0]
    x, y, w, h = boxes[idx]
    label = '{} : {:.3f}'.format(str(classes[classes_ids[idx]]), confidences[idx])

    color = colors[idx]

    car_img = cv2.rectangle(img, (x, y), (x+w, y+h), color, 3)
    cv2.putText(img, label, (x, y - 10), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 1)
    
    carROI = gray[y:y+h, x:x+w]
    car_nums = cascade.detectMultiScale(carROI)
    for (x2, y2, w2, h2) in car_nums:
        car_img = cv2.rectangle(car_img, (x + x2, y + y2), (x + x2 + w2, y + y2+ h2), (0, 255, 255), 3)

    cv2.imshow('car', car_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()   

In [11]:
cv2.imwrite('car-fin.jpg', car_img)

True

## Car 실습 - Simpson

In [26]:
import numpy as np
import cv2

# 불러오기
file_name = './simpson.jpg'
img = cv2.imread(file_name)

yolo_weights = './darknet/weights/yolov3.weights'
yolo_names = './darknet/data/coco.names'
yolo_cfg = './darknet/cfg/yolov3.cfg'

# 모델 불러오기
net = cv2.dnn.readNet(yolo_weights, yolo_cfg)
output_layers = net.getUnconnectedOutLayersNames()

# 모델 세팅
classes = []
with open(yolo_names, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    
colors = np.random.uniform(0, 255, size=(len(classes), 3))
min_confidence = 0.5
height, width, channels = img.shape

blob = cv2.dnn.blobFromImage(img,
                             scalefactor=1/255.,
                             size=(416, 416),
                             mean=(0, 0, 0),
                             swapRB=True,
                             crop=False)

In [27]:
net.setInput(blob)
outs = net.forward(output_layers)

In [28]:
class_ids = []
boxes = []
confidences = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        
        if min_confidence < confidence:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x + w//2)
            y = int(center_y + h//2)
            
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [29]:
idxes = cv2.dnn.NMSBoxes(boxes,
                        confidences,
                        min_confidence,
                        0.4)

for i in idxes:
    idx = i[0]
    x, y, w, h = boxes[idx]
    
    color = colors[idx]
    label = f'{str(classes[class_ids[idx]])}, {confidences[idx]:.3f}'
    
    cv2.rectangle(img, (x, y), (x+w, y+h), color, 3)
    cv2.putText(img, label, (x, y-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, color, 1)
    
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [30]:
cv2.imwrite('car-simpson-traffic-congestion.jpg', img)

True